## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-16-02-25-38 +0000,nypost,"Breast cancer surviving grandma, 42, hits hist...",https://nypost.com/2025/08/15/us-news/breast-c...
1,2025-08-16-02-15-54 +0000,nyt,Trump Administration Backs Off New Attempt to ...,https://www.nytimes.com/2025/08/15/us/washingt...
2,2025-08-16-02-13-04 +0000,nyt,6 Takeaways From Trump’s Meeting With Putin,https://www.nytimes.com/2025/08/15/us/politics...
3,2025-08-16-02-12-01 +0000,nyt,Live Updates: Trump Says There Is ‘No Deal’ on...,https://www.nytimes.com/live/2025/08/15/world/...
4,2025-08-16-02-11-59 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/08/15/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2380/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,68
22,putin,41
28,ukraine,19
29,summit,17
15,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
154,2025-08-15-15-56-48 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...,216
181,2025-08-15-13-25-54 +0000,nypost,Putin’s foreign minister trolls Ukraine by wea...,https://nypost.com/2025/08/15/world-news/putin...,204
257,2025-08-15-06-37-53 +0000,cbc,Trump and Putin end Alaska summit without deal...,https://www.cbc.ca/9.6867437?cmp=rss,175
27,2025-08-16-00-14-50 +0000,nypost,"Putin tries to woo Trump at Alaska meeting, cl...",https://nypost.com/2025/08/15/us-news/putin-tr...,170
10,2025-08-16-01-50-10 +0000,bbc,"No ceasefire, no deal. What summit means for T...",https://www.bbc.com/news/articles/clyvd3gkg1po...,168


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
154,216,2025-08-15-15-56-48 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...
262,53,2025-08-15-04-27-14 +0000,nypost,AG Pam Bondi declares DC ‘will not remain a sa...,https://nypost.com/2025/08/15/us-news/ag-pam-b...
193,52,2025-08-15-12-35-18 +0000,nypost,Hillary Clinton admits she would nominate Trum...,https://nypost.com/2025/08/15/us-news/hillary-...
1,46,2025-08-16-02-15-54 +0000,nyt,Trump Administration Backs Off New Attempt to ...,https://www.nytimes.com/2025/08/15/us/washingt...
10,40,2025-08-16-01-50-10 +0000,bbc,"No ceasefire, no deal. What summit means for T...",https://www.bbc.com/news/articles/clyvd3gkg1po...
61,31,2025-08-15-21-57-52 +0000,nypost,ICE nabs illegal immigrant accused of killing ...,https://nypost.com/2025/08/15/us-news/ice-nabs...
124,30,2025-08-15-17-37-26 +0000,nypost,Ukraine’s first lady demands return of thousan...,https://nypost.com/2025/08/15/world-news/ukrai...
117,28,2025-08-15-18-27-02 +0000,nypost,Feds accuse New Orleans Mayor LaToya Cantrell ...,https://nypost.com/2025/08/15/us-news/mayor-la...
241,27,2025-08-15-08-22-31 +0000,bbc,"My family may be killed if deported, says son ...",https://www.bbc.com/news/articles/c776zgj73lpo...
92,26,2025-08-15-20-41-15 +0000,latimes,Playboy leaves Los Angeles for Miami Beach; CE...,https://www.latimes.com/business/story/2025-08...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
